In [1]:
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
% matplotlib inline
import time
import datetime

In [2]:
#load data dictionary from pkl file
iontype = "Ca"
SM_NUM = 2
sm_str = "SM"+str(SM_NUM)
NORM_MEDIAN = True

if NORM_MEDIAN:
    extra_str = "_MED_"
else:
    extra_str = ""

# load data
f = open("/media/icg-channels/icg-channels-"+iontype+".pkl","rb")
data_dict = pickle.load(f)

# immediately save a backup just in case
st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d-%H-%M-%S')
f = open("/media/icg-channels/backup/icg-channels-"+iontype+"-"+st+".pkl",'wb')
pickle.dump(data_dict,f)

In [3]:
# get list of normal trace files and sm_trace files

tracedir = "/media/icg-channels/icg-channels-"+iontype+"/"
protocols = ['act','deact','inact','ramp','ap']

orig_traces = []
sm_traces =[]
for (dirpath, dirnames, filenames) in os.walk(tracedir):
    if 'ICG_'+sm_str in dirpath:
        for file in filenames:
            if ".i" in file and ("sm"+str(SM_NUM)+"_" in file):
                sm_traces.append(file)
    if 'ICG_ORIG' in dirpath:
        for file in filenames:
            if ".i" in file:
                orig_traces.append(file)

orig_traces_unique = orig_traces.copy()
for i in range(len(orig_traces)):
    for p in protocols:
        orig_traces_unique[i] = orig_traces_unique[i].replace('_'+p+'.i','.mod')
orig_traces_unique = sorted(list(set(orig_traces_unique)))

sm_traces_unique = sm_traces.copy()
for i in range(len(sm_traces)):
    for p in protocols:
        sm_traces_unique[i] = sm_traces_unique[i].replace('_'+p+'.i','.mod')
        sm_traces_unique[i] = sm_traces_unique[i].replace("sm"+str(SM_NUM)+'_','')
sm_traces_unique = sorted(list(set(sm_traces_unique)))

print("Ion type: ",iontype)
print("Number of original file traces: ",len(orig_traces_unique))
print("Number of supermodel file traces: ", len(sm_traces_unique))

Ion type:  Ca
Number of original file traces:  524
Number of supermodel file traces:  387


In [4]:
# which files are missing?
sm_missing = [f for f in orig_traces_unique if f not in sm_traces_unique]
orig_missing = [f for f in sm_traces_unique if f not in orig_traces_unique]

# NOTE: uncomment this to print list of missing files
#print("Missing supermodel files:")
#print(sm_missing)
#print("Missing original files:")
#print(orig_missing)

## main loop -- load ICG traces for pairs of traces (original and supermodel) and calculate RMSE
### NOTE: this takes a while to run (5-10 seconds per model)

In [5]:
# collect rms (and area?) statistics
for i in range(len(orig_traces_unique)):
    otr = orig_traces_unique[i]
    data_dict[otr]['ICG_ORIG']= True
    
    sm_inds = [j for j,x in enumerate(sm_traces_unique) if otr in x]
    if not sm_inds:
        print(i, ': supermodel traces not found')
        data_dict[otr]['ICG_'+sm_str+extra_str]= False
        continue
    else:
        print(i)
        data_dict[otr]['ICG_'+sm_str+extra_str]= True
        data_dict[otr]['ICG_'+sm_str+extra_str+'_ERROR1'] = {}
        data_dict[otr]['ICG_'+sm_str+extra_str+'_ERROR3'] = {}

    full_orig_data = np.array([])
    full_sm_data = np.array([])
    mse = 0.0
    for j in range(len(protocols)):
        p = protocols[j]
        orig_data = np.loadtxt(tracedir+'/'+otr+'/ICG_ORIG/'+otr.replace('.mod','_'+p+'.i'))
        sm_data = np.loadtxt(tracedir+'/'+otr+'/ICG_'+sm_str+'/sm'+str(SM_NUM)+'_'+otr.replace('.mod','_'+p+'.i'))
        
        # renormalize by the median??
        orig_data = orig_data / np.median(orig_data)
        sm_data = sm_data / np.median(sm_data)
        
        # take only every 10th data point and (don't) renormalize
        orig_data = orig_data[::10,:]
        #orig_data[:,1:] = orig_data[:,1:]/np.max(orig_data[:,1:])
        sm_data = sm_data[::10,:]
        #sm_data[:,1:] = sm_data[:,1:]/np.max(sm_data[:,1:])
        
        mse_tmp = np.sum(np.square(orig_data[:,1:]-sm_data[:,1:]))
        tss_tmp = np.sum(np.square(orig_data[:,1:]-np.mean(orig_data[:,1:])))
        data_dict[otr]['ICG_'+sm_str+extra_str+'_ERROR1'][p] = np.sqrt(mse_tmp/tss_tmp)
        data_dict[otr]['ICG_'+sm_str+extra_str+'_ERROR3'][p] = np.linalg.norm(orig_data[:,1:]-sm_data[:,1:])/np.linalg.norm(orig_data[:,1:])
        mse += mse_tmp
        full_orig_data= np.concatenate((full_orig_data,orig_data[:,1:].flatten()),axis=0)
        full_sm_data= np.concatenate((full_sm_data,sm_data[:,1:].flatten()),axis=0)
    tss_mean = np.mean(full_orig_data)
    tss = np.sum(np.square(full_orig_data - tss_mean))
    data_dict[otr]['ICG_'+sm_str+extra_str+'_ERROR1']['total'] = np.sqrt(mse/tss)
    data_dict[otr]['ICG_'+sm_str+extra_str+'_ERROR3']['total'] = np.linalg.norm(full_orig_data-full_sm_data)/np.linalg.norm(full_orig_data)


0
1
2
3
4
5
6
7
8
9
10
11
12 : supermodel traces not found
13
14
15
16
17
18
19
20
21
22 : supermodel traces not found
23 : supermodel traces not found
24
25
26
27
28
29
30
31
32
33 : supermodel traces not found
34 : supermodel traces not found
35
36
37
38
39
40
41 : supermodel traces not found
42 : supermodel traces not found
43
44
45
46
47
48
49
50
51
52 : supermodel traces not found
53 : supermodel traces not found
54
55
56
57
58
59
60
61 : supermodel traces not found
62 : supermodel traces not found
63
64
65
66
67
68
69
70
71
72 : supermodel traces not found
73
74 : supermodel traces not found
75
76
77 : supermodel traces not found
78
79
80
81
82
83 : supermodel traces not found
84
85
86
87 : supermodel traces not found
88 : supermodel traces not found
89 : supermodel traces not found
90
91
92
93 : supermodel traces not found
94 : supermodel traces not found
95 : supermodel traces not found
96
97
98
99 : supermodel traces not found
100 : supermodel traces not found
101 : supermodel

/opt/anaconda/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


458
459
460
461
462
463
464 : supermodel traces not found
465 : supermodel traces not found
466 : supermodel traces not found
467
468
469
470 : supermodel traces not found
471
472
473
474 : supermodel traces not found
475 : supermodel traces not found
476 : supermodel traces not found
477
478
479
480
481 : supermodel traces not found
482 : supermodel traces not found
483 : supermodel traces not found
484 : supermodel traces not found
485 : supermodel traces not found
486 : supermodel traces not found
487
488 : supermodel traces not found
489 : supermodel traces not found
490
491 : supermodel traces not found
492 : supermodel traces not found
493
494
495
496
497
498 : supermodel traces not found
499 : supermodel traces not found
500 : supermodel traces not found
501 : supermodel traces not found
502 : supermodel traces not found
503 : supermodel traces not found
504
505
506
507
508
509
510
511 : supermodel traces not found
512 : supermodel traces not found
513
514
515
516
517 : supermod

In [6]:
# re-save data dict
st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d-%H-%M-%S')
f = open("/media/icg-channels/backup/icg-channels-"+iontype+"-"+st+".pkl",'wb')
pickle.dump(data_dict,f)
f = open("/media/icg-channels/icg-channels-"+iontype+".pkl",'wb')
pickle.dump(data_dict,f)